# Introduction

These are SQL notes that I frequently refer to. They assume a certain level of familiarity with SQL, do not explicitly call out the nuances, and maybe missing some key concepts. The flavor of SQL used here is suited for SQL-Server and other databases might have a slight variation of the actual syntax. 

There are several sources for learning SQL on the web. Here are couple of good ones
1) [SQLBolt](https://sqlbolt.com/)
2) [Geeks for Geeks](https://www.geeksforgeeks.org/sql-join-cartesian-join-self-join/).  

Happy referencing :thumbsup:

------------------------------------------------
**1) Getting started**
------------------------------------------------
**Column Types**
```
SELECT COLUMN_NAME,
       DATA_TYPE,
       CHARACTER_MAXIMUM_LENGTH
FROM information_schema.columns
WHERE TABLE_NAME = 'your_table_name'
```
**Limit and Offset**
```
SELECT title FROM movies
ORDER BY title ASC
LIMIT 5 OFFSET 5
```
------------------------------------------------
**2) Order of execution for a "Select" query**
------------------------------------------------
```
SELECT DISTINCT column, AGG_FUNC(column_or_expression), …
FROM mytable
JOIN another_table ON mytable.column = another_table.column
WHERE constraint_expression
GROUP BY column
HAVING constraint_expression
ORDER BY column ASC/DESC
LIMIT count OFFSET COUNT;
```
**Here is how this gets executed**  

1. FROM and JOINs  
The FROM clause, and subsequent JOINs are first executed to determine the total working set of data that is being queried. This includes subqueries in this clause, and can cause temporary tables to be created under the hood containing all the columns and rows of the tables being joined.
2. WHERE  
Once we have the total working set of data, the first-pass WHERE constraints are applied to the individual rows, and rows that do not satisfy the constraint are discarded. Each of the constraints can only access columns directly from the tables requested in the FROM clause. Aliases in the SELECT part of the query are not accessible in most databases since they may include expressions dependent on parts of the query that have not yet executed.
3. GROUP BY  
The remaining rows after the WHERE constraints are applied are then grouped based on common values in the column specified in the GROUP BY clause. As a result of the grouping, there will only be as many rows as there are unique values in that column. Implicitly, this means that you should only need to use this when you have aggregate functions in your query.
4. HAVING  
If the query has a GROUP BY clause, then the constraints in the HAVING clause are then applied to the grouped rows, discard the grouped rows that don't satisfy the constraint. Like the WHERE clause, aliases are also not accessible from this step in most databases.
5. SELECT  
Any expressions in the SELECT part of the query are finally computed.
6. DISTINCT  
Of the remaining rows, rows with duplicate values in the column marked as DISTINCT will be discarded.
7. ORDER BY  
If an order is specified by the ORDER BY clause, the rows are then sorted by the specified data in either ascending or descending order. Since all the expressions in the SELECT part of the query have been computed, you can reference aliases in this clause.
8. LIMIT / OFFSET  
Finally, the rows that fall outside the range specified by the LIMIT and OFFSET are discarded, leaving the final set of rows to be returned from the query.
------------------------------------------------
**3) Joins**
------------------------------------------------
**Inner Join**
```
SELECT table1.*,table2.*
FROM table1 
INNER JOIN table2
ON table1.matching_column = table2.matching_column;
```

Very often, the term "Inner" is omitted and we just write
```
SELECT table1.*,table2.*
FROM table1 
JOIN table2
ON table1.matching_column = table2.matching_column;
```
<img src="https://user-images.githubusercontent.com/92488889/222541266-144a4c68-0ff0-4a63-ac7b-b88a12301206.png" width="502" height="329">

**Left Join**
```
SELECT table1.*,table2.*
FROM table1 
LEFT JOIN table2
ON table1.matching_column = table2.matching_column;
```
<img src="https://user-images.githubusercontent.com/92488889/222541296-09afaea5-dd66-4a14-ac36-fa8d4c0c317f.png" width="502" height="329">

**Right Join**
```
SELECT table1.*,table2.*
FROM table1 
RIGHT JOIN table2
ON table1.matching_column = table2.matching_column;
```
<img src="https://user-images.githubusercontent.com/92488889/222541323-2557658a-fc5c-4f0a-8642-8bce740943fc.png" width="550" height="360">

**Full/Outer Join**
```
SELECT table1.*,table2.*
FROM table1 
FULL JOIN table2
ON table1.matching_column = table2.matching_column;
```
<img src="https://user-images.githubusercontent.com/92488889/222541148-0f3139e7-b3b6-4b31-b92e-224efcccb499.png" width="502" height="329">

[Image Source](https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/)


------------------------------------------------
**4) Table Operations**
------------------------------------------------

**Create a new table**  
Here are several example of how to create a new table

**Creating a dummy table**
```
-- Example 1
CREATE TABLE IF NOT EXISTS mytable 
(
    column1 DataType TableConstraint DEFAULT default_value,
    column2 DataType TableConstraint DEFAULT default_value,
    …
);  
-- Example 2
CREATE TABLE movies (
    id INTEGER PRIMARY KEY,
    title TEXT,
    director TEXT,
    year INTEGER, 
    length_minutes INTEGER
);
```
**Deriving a table from an existing table**
```
CREATE TABLE IF NOT EXISTS mytable 
(
       SELECT * FROM existing_table
       WHERE existing_table_sales between 1000 and 2000 
);
```
**Add columns to table**
```
ALTER TABLE mytable
ADD column DataType OptionalTableConstraint 
    DEFAULT default_value;
```
**Removing columns in existing table**
```
ALTER TABLE mytable
DROP column_to_be_dropped;
```
**Rename columns in table**
```
ALTER TABLE mytable
RENAME COLUMN OldColumnName TO NewColumnName;
```
**Add rows to table**
```
UPDATE mytable
SET column = value_or_expr, 
    other_column = another_value_or_expr, 
    …
WHERE condition;
```
**Drop rows to table**
```
DELETE FROM mytable
WHERE condition;
```
**Rename table**
```
ALTER TABLE table_name
RENAME TO new_table_name;
```

**Drop table**
```
DROP TABLE IF EXISTS mytable;
```
------------------------------------------------
**5) CTEs to organize complex queries**
------------------------------------------------
Common Table Expressions or CTEs can be very effective in organizing queries which otherwise will need to be handled as nested queries which can become bloated and less readable.  
Here is an example
```
WITH cte_category_counts (category_id, category_name, product_count) AS 
(
    SELECT 
        c.category_id, 
        c.category_name, 
        COUNT(p.product_id)
    FROM 
        production.products p
        INNER JOIN production.categories c 
            ON c.category_id = p.category_id
    GROUP BY 
        c.category_id, 
        c.category_name
),
cte_category_sales(category_id, sales) AS 
(
    SELECT    
        p.category_id, 
        SUM(i.quantity * i.list_price * (1 - i.discount))
    FROM    
        sales.order_items i
        INNER JOIN production.products p 
            ON p.product_id = i.product_id
        INNER JOIN sales.orders o 
            ON o.order_id = i.order_id
    WHERE order_status = 4 -- completed
    GROUP BY 
        p.category_id
) 

SELECT 
    c.category_id, 
    c.category_name, 
    c.product_count, 
    s.sales
FROM
    cte_category_counts c
    INNER JOIN cte_category_sales s 
        ON s.category_id = c.category_id
ORDER BY 
    c.category_name;
```
Note that anything that is declared as a CTE needs to be used in the query that follows.

------------------------------------------------
**6) Window Functions**
------------------------------------------------
In SQL, a window function or analytic function is a function which uses values from one or multiple rows to return a value for each row. (This contrasts with an aggregate function, which returns a single value for multiple rows.) Window functions have an OVER clause; any function without an OVER clause is not a window function, but rather an aggregate or single-row (scalar) function.
```
-- Example 1 
-- Avg. salary by department
SELECT Name, Age, Department, Salary, 
AVERAGE(Salary) OVER( PARTITION BY Department ORDER BY Age) AS Avg_Salary
FROM employee;  

-- Example 2 
-- 3 day moving avg. price  
select 
record_date,
avg(price) over(order by record_date rows between 2 preceding and current row) 
from company_price_data;
```
Here is a more elaborate list
<img src="https://user-images.githubusercontent.com/92488889/222567377-711a52eb-866e-4faf-88a2-3369cb73138e.png" width="800" height="550">  
[Image Source](https://tinylarry.com/guide-to-sql-window-functions-when-and-how-to-use-them/)

------------------------------------------------
**7) Table Vs. View**
------------------------------------------------

**Table:**  
Table is a preliminary storage for storing data and information in RDBMS. A table is a collection of related data entries, and it consists of columns and rows.  

**View:**  
A view is a virtual table whose contents are defined by a query. Unless indexed, a view does not exist as a stored set of data values in a database.  

Advantages over table are
* We can combine columns/rows from multiple table or another view and have a consolidated view.
* Views can be used as security mechanisms by letting users access data through the view, without granting the users permissions to directly access the underlying base tables of the view
* It acts as abstract layer to downstream systems, so any change in schema is not exposed and hence the downstream systems doesn't get affected.

In view there is not any direct or physical relation with the database. And Modification through 
a view (e.g. insert, update, delete) is not permitted.Its just a logical set of tables. In layman’s terms, a view is a “stored query “.

------------------------------------------------
**8) Changing variable types**
------------------------------------------------
The CAST() function converts a value (of any type) into a specified datatype.
```
SELECT CAST('2017-08-25' AS datetime);  

SELECT 1 + '1' AS result;  
-- Implicitly text converts to number
-- Ans: 2

SELECT 2 + CAST('1' AS INT) result;
-- Explicitly converting text to a number
-- Ans: 3
```
------------------------------------------------
**9) Date manipulations**
------------------------------------------------
The following lists the most important built-in date functions
```
GETDATE()
--Returns the current date and time

DATEPART()
--Returns a single part of a date/time

DATEADD()
--Adds or subtracts a specified time interval from a date

DATEDIFF()
--Returns the time between two dates

CONVERT()
--Displays date/time data in different formats
````
I would also highly recommend creating a Date Dimension Table (if you already don't have one in your DB). You can find some sample code you can use [here.](https://gist.github.com/anuplotter/b21b31d54a7d948c108fedab59c9bdd7)

------------------------------------------------
**10) OLTP vs. OLAP**
------------------------------------------------

The two terms look similar but refer to different kinds of systems. **Online transaction processing (OLTP)** captures, stores, and processes data from transactions in real time. **Online analytical processing (OLAP)** uses complex queries to analyze aggregated historical data from OLTP systems.  

**What is OLTP?**
An OLTP system captures and maintains transaction data in a database. Each transaction involves individual database records made up of multiple fields or columns. Examples include banking and credit card activity or retail checkout scanning.
In OLTP, the emphasis is on fast processing, because OLTP databases are read, written, and updated frequently. If a transaction fails, built-in system logic ensures data integrity.  

**What is OLAP?**
OLAP applies complex queries to large amounts of historical data, aggregated from OLTP databases and other sources, for data mining, analytics, and business intelligence projects. In OLAP, the emphasis is on response time to these complex queries. Each query involves one or more columns of data aggregated from many rows. Examples include year-over-year financial performance or marketing lead generation trends. OLAP databases and data warehouses give analysts and decision-makers the ability to use custom reporting tools to turn data into information. Query failure in OLAP does not interrupt or delay transaction processing for customers, but it can delay or impact the accuracy of business intelligence insights.
